
## The Decision Problem
We make widgets. Have a set of production facilities that produce boxes of widgets. There is also a set of distribution locations that will then distribute the widgets for sale. Each distribution center has a forecasted demand and each production facility has a min and max number of widgets it can make during this period. We need to ensure that each distribution facility receives enough widgets to satisfy demand from production and we want to do this at minimal cost. The minimum production is 75% of the production facilities max value. 

Our sets are:
- $P = \{\texttt{'Baltimore', 'Cleveland', 'Little Rock', 'Birmingham', 'Charleston'}\} \quad\quad\quad\quad\quad\quad\quad\space\space \texttt{Production}$
- $D = \{\texttt{'Columbia', 'Indianapolis', 'Lexington', 'Nashville', 'Richmond', 'St. Louis'}\} \quad\quad \texttt{Distribution}$


In [49]:
# Import important packages 
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Sets P and D, respectively
production = ['Baltimore','Cleveland','Little Rock','Birmingham','Charleston']
distribution = ['Columbia','Indianapolis','Lexington','Nashville','Richmond','St. Louis']

# Define a gurobipy model for the decision problem
model = gp.Model('widgets')

Restricted license - for non-production use only - expires 2025-11-24


## Parameters

- $m_p$ is the max production in location $p$, $\forall p \in P \quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\space\space \texttt{max}\_\texttt{prod[p]}$
- $n_d$ is the number of customers for a distribution center $d$, $\forall d \in D \quad\quad\quad\quad\quad\quad\quad\quad \texttt{n}\_\texttt{demand[d]}$
- $c_{p,d}$ is the cost to ship a widget between location $p$ and location $d$, $\forall p \in P, d \in D \quad\quad\quad \texttt{cost[p,d]}$

In [50]:
# Import Dataset 
path = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/Modeling_Session_1/cost.csv'
transp_cost = pd.Series(pd.read_csv(path).groupby(["production","distribution"])["cost"].sum())
transp_cost.reset_index().pivot(index='production', columns='distribution', values='cost')

distribution,Columbia,Indianapolis,Lexington,Nashville,Richmond,St. Louis
production,,,,,,
Baltimore,4.50,5.09,4.33,5.96,1.96,7.30
Birmingham,3.33,4.33,3.38,1.53,5.95,4.01
Charleston,3.02,2.61,1.61,4.44,2.36,4.60
Cleveland,2.43,2.37,2.54,4.13,3.20,4.88
Little Rock,6.42,4.83,3.39,4.40,7.44,2.92


In [51]:
max_prod = pd.Series([180,200,140,80,180], index = production, name = "max_production")
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name = "demand") 
max_prod.to_frame()

,max_production
Baltimore,180
Cleveland,200
Little Rock,140
Birmingham,80
Charleston,180


We also have the requirement that each production facility needs to produce at 75% of this maximum output. We'll denote this value by $a$ in the formulation "frac" for the fraction of maximum production required. Initially we set $a = 0.75$.

In [52]:
frac = 0.75

In [53]:
# Add Variables 
x = model.addVars(production, distribution, name = 'prod_ship')
model.update()
x

{('Baltimore', 'Columbia'): <gurobi.Var prod_ship[Baltimore,Columbia]>,
 ('Baltimore', 'Indianapolis'): <gurobi.Var prod_ship[Baltimore,Indianapolis]>,
 ('Baltimore', 'Lexington'): <gurobi.Var prod_ship[Baltimore,Lexington]>,
 ('Baltimore', 'Nashville'): <gurobi.Var prod_ship[Baltimore,Nashville]>,
 ('Baltimore', 'Richmond'): <gurobi.Var prod_ship[Baltimore,Richmond]>,
 ('Baltimore', 'St. Louis'): <gurobi.Var prod_ship[Baltimore,St. Louis]>,
 ('Cleveland', 'Columbia'): <gurobi.Var prod_ship[Cleveland,Columbia]>,
 ('Cleveland', 'Indianapolis'): <gurobi.Var prod_ship[Cleveland,Indianapolis]>,
 ('Cleveland', 'Lexington'): <gurobi.Var prod_ship[Cleveland,Lexington]>,
 ('Cleveland', 'Nashville'): <gurobi.Var prod_ship[Cleveland,Nashville]>,
 ('Cleveland', 'Richmond'): <gurobi.Var prod_ship[Cleveland,Richmond]>,
 ('Cleveland', 'St. Louis'): <gurobi.Var prod_ship[Cleveland,St. Louis]>,
 ('Little Rock', 'Columbia'): <gurobi.Var prod_ship[Little Rock,Columbia]>,
 ('Little Rock',
  'Indianapolis

## Constraints

### Our Constraints
To start, we'll formulate the demand constraints for each distribution location first and add them to the model.

\begin{align*} 
\sum_{p}x_{p,d} \ge n_d, \quad \forall d \in D \quad\quad \texttt{meet}\_\texttt{demand[d]}\\ 
\end{align*}
Next we have the maximum number of widgets each production facility can make. We also have that each facility must make at least 75% of its max production. 

$$
\begin{align*} 
\sum_{d}x_{p,d} &\le m_p, &\forall p \in P \quad\quad &\texttt{can}\_\texttt{produce[p]}\\ 
\sum_{d}x_{p,d} &\ge a*m_p,&\forall p \in P \quad\quad &\texttt{must}\_\texttt{produce[p]}\\ 
\end{align*}
$$


In [54]:
meet_demand = model.addConstrs((gp.quicksum(x[p,d] for p in production) >= n_demand[d] for d in distribution), 
                               name = 'meet_demand')
can_produce = model.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production), 
                               name = 'can_produce')# (x.sum(i, '*')
must_produce = model.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production), 
                                name = 'must_produce')
model.update()
can_produce

{'Baltimore': <gurobi.Constr can_produce[Baltimore]>,
 'Cleveland': <gurobi.Constr can_produce[Cleveland]>,
 'Little Rock': <gurobi.Constr can_produce[Little Rock]>,
 'Birmingham': <gurobi.Constr can_produce[Birmingham]>,
 'Charleston': <gurobi.Constr can_produce[Charleston]>}

## Objective Function
We were told to (Reduce) the transportation costs and we'll use this to determine our objective function as minimizing the total cost to ship widgets from production to distribution locations. 
### Our Objective Function
$$
\begin{align*}
\text{minimize} \quad & \sum_{p \in P} \sum_{d \in D} c_{p,d} x_{p,d}, \\
& \forall p \in P, \forall d \in D
\end{align*}


In [55]:
model.setObjective(gp.quicksum(transp_cost[p,d]*x[p,d] for p in production for d in distribution), GRB.MINIMIZE) 

## Find, Extract, and Analyze the Solution

In [56]:
model.write('widget_shipment.lp')

### Run the Optimization

In [57]:
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: AMD EPYC 7763 64-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 16 rows, 30 columns and 90 nonzeros
Model fingerprint: 0x20186c14
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 2e+02]
Presolve removed 5 rows and 0 columns
Presolve time: 0.01s
Presolved: 11 rows, 35 columns, 65 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.610000e+02   0.000000e+00      0s
      15    1.7048900e+03   0.000000e+00   0.000000e+00      0s

Solved in 15 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.704890000e+03


### Extract the Solution

In [58]:
x_values = pd.Series(model.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol = pd.concat([transp_cost, x_values], axis=1)
#sol 
sol

cost  shipment
production  distribution                
Baltimore   Columbia      4.50       0.0
            Indianapolis  5.09       0.0
            Lexington     4.33       0.0
            Nashville     5.96      19.0
            Richmond      1.96     116.0
            St. Louis     7.30       0.0
Birmingham  Columbia      3.33       0.0
            Indianapolis  4.33       0.0
            Lexington     3.38       0.0
            Nashville     1.53      80.0
            Richmond      5.95       0.0
            St. Louis     4.01       0.0
Charleston  Columbia      3.02       0.0
            Indianapolis  2.61       0.0
            Lexington     1.61     121.0
            Nashville     4.44       0.0
            Richmond      2.36       0.0
            St. Louis     4.60      41.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
            Lexington     2.54       0.0
            Nashville     4.13       2.0
            Richmond      3.20       0.0
            St. Louis     4.88       0.0
Little Rock Columbia      6.42       0.0
            Indianapolis  4.83       0.0
            Lexington     3.39       0.0
            Nashville     4.40       0.0
            Richmond      7.44       0.0
            St. Louis     2.92     140.0

### Solution Analysis
By aggregating the total production by facility to see which locations did not produce their maximum capacity of widgets and which production facilities are at the lower bound of their production. 

In [59]:
# Sum the shipment amount by production facility
ship_out = sol.groupby('production')['shipment'].sum()
pd.DataFrame({"max_prod":max_prod,"Opt_ship_out":ship_out,'Remaining':max_prod - ship_out, 'Utilization':ship_out/max_prod})

,max_prod,Opt_ship_out,Remaining,Utilization
Baltimore,180,135.0,45.0,0.75
Birmingham,80,80.0,0.0,1.00
Charleston,180,162.0,18.0,0.90
Cleveland,200,186.0,14.0,0.93
Little Rock,140,140.0,0.0,1.00


# Using Binary Variables
In the original problem Birmingham's production is much lower than the rest of the facilities. Suppose we have the option to expand that facilities max capacity by either 25 or 50 widgets, but there is a cost of \\$50 and \\$75, respectively, to choose one of these options and we can choose at most one. We'll use a binary decision variable for each option named $xprod$. 

Let $xprod_0 = 1$ if we choose the first option and expand production capacity by 25 and $0$ otherwise.
Let $xprod_1 = 1$ if we choose the second option and expand production capacity by 50 and $0$ otherwise.


In [60]:
# These parts are the same as above outside of the new model name
model2 = gp.Model('widgets2')
x = model2.addVars(production, distribution, obj = transp_cost, name = 'prod_ship')
meet_demand = model2.addConstrs((gp.quicksum(x[p,d] for p in production) >= n_demand[d] for d in distribution), name = 'meet_demand')

### Add Constraints 
$$
\begin{align*} 
\sum_{d}x_{p,d} &\le m_p, &\forall p \in P -\{\texttt{Birmingham}\} \\ 
\sum_{d}x_{p,d} &\ge a*m_p,&\forall p \in P -\{\texttt{Birmingham}\} \\ 
\end{align*}
$$


In [61]:
can_produce = model2.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production if p != 'Birmingham'), name = 'can_produce')
must_produce =  model2.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production if p != 'Birmingham'), name = 'must_produce')

Now, add the new binary variables.

In [62]:
xprod = model2.addVars(range(2), vtype = GRB.BINARY, obj = [50, 75], name = 'expand_Birmingham_prod')


The objective and new binary variables look like this in the formulation: 

\begin{align*} 
{\rm minimize} \space &\sum_{p,d}c_{p,d}x_{p,d} + 50*xprod_0 + 75*xprod_1, \quad &\forall p \in P, d \in D\\ 
&xprod_i \in \{0,1\}, &{\rm for} \space i \in \{0,1\}
\end{align*}

Next we have the production constraints that are specific to the Birmingham facility.

$$
\begin{align*} 
\sum_{d}x_{p,d} &\le m_p + 25*xprod_0 + 50*xprod_1, & p = \texttt{Birmingham} \\ 
\sum_{d}x_{p,d} &\ge a*(m_p+ 25*xprod_0 + 50*xprod_1),& p = \texttt{Birmingham} \\ 
\end{align*}
$$

In [63]:
Birmingham_max = model2.addConstr(gp.quicksum(x['Birmingham',d] for d in distribution) <= max_prod['Birmingham'] + 25*xprod[0] + 50*xprod[1], name = 'Birmingham_add_max')
Birmingham_min = model2.addConstr(gp.quicksum(x['Birmingham',d] for d in distribution) >= frac*(max_prod['Birmingham'] + 25*xprod[0] + 50*xprod[1]), name = 'Birmingham_add_min')

It was stated above that we can select at most one of the expansion options which means we cannot allow both $xprod_0$ and $xprod_1$ to equal one. To model this we add a constraint limiting the sum of these two binary variables to at most one.

$$
\begin{align*}
\sum_{i}xprod_i \le 1
\end{align*}
$$


In [64]:
Birmingham_lim = model2.addConstr(gp.quicksum(xprod[i] for i in range(2)) <= 1, name = 'expansion_choice')

In [65]:
model2.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: AMD EPYC 7763 64-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 17 rows, 32 columns and 96 nonzeros
Model fingerprint: 0x76de185e
Variable types: 30 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [2e+00, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve time: 0.00s
Presolved: 17 rows, 32 columns, 96 nonzeros
Variable types: 30 continuous, 2 integer (2 binary)
Found heuristic solution: objective 1651.0200000

Root relaxation: cutoff, 10 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      1651.02000 1651.02000  0.00%     -    0s

In [66]:
obj1 = model.getObjective()
obj2 = model2.getObjective()
print(f"The original model had a total cost of {round(obj1.getValue(),2)}")
print(f"The new formualtion has a total cost of {round(obj2.getValue(),2)}")

The original model had a total cost of 1704.89
The new formualtion has a total cost of 1651.02


What does the change in objective function value tell us? 


In [67]:
pd.Series(model2.getAttr('X', xprod))

0   -0.0
1   -0.0
dtype: float64

The model selected the first expansion option since $xprod_0 = 1$, which was increasing production by 25 widgets in Birmingham. We can see the rest of the solution, which will include the increase in Birmingham's production capacity. 

In [70]:
x2_values = pd.Series(model2.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol2 = pd.concat([transp_cost, x2_values], axis=1)
sol2[sol2.shipment > 0]

cost  shipment
production  distribution                
Baltimore   Columbia      4.50      19.0
            Richmond      1.96     116.0
Birmingham  Columbia      3.33       4.0
            Indianapolis  4.33      76.0
Charleston  St. Louis     4.60     180.0
Cleveland   Columbia      2.43      66.0
            Nashville     4.13     101.0
            St. Louis     4.88       1.0
Little Rock Indianapolis  4.83      19.0
            Lexington     3.39     121.0

In [69]:
ship_out = sol2.groupby('production')['shipment'].sum()
pd.DataFrame({"max_prod":max_prod,"Opt_ship_out":ship_out,'Remaining':max_prod - ship_out, 'Utilization':ship_out/max_prod})

,max_prod,Opt_ship_out,Remaining,Utilization
Baltimore,180,135.0,45.0,0.75
Birmingham,80,80.0,0.0,1.00
Charleston,180,180.0,0.0,1.00
Cleveland,200,168.0,32.0,0.84
Little Rock,140,140.0,0.0,1.00


In [71]:
ship_out = sol.groupby('production')['shipment'].sum()
pd.DataFrame({"max_prod":max_prod,"Opt_ship_out":ship_out,'Remaining':max_prod - ship_out, 'Utilization':ship_out/max_prod})

,max_prod,Opt_ship_out,Remaining,Utilization
Baltimore,180,135.0,45.0,0.75
Birmingham,80,80.0,0.0,1.00
Charleston,180,162.0,18.0,0.90
Cleveland,200,186.0,14.0,0.93
Little Rock,140,140.0,0.0,1.00
